In [ ]:
from google.colab import drive
from google.colab import files

In [ ]:
!mkdir GD

In [ ]:
!mkdir GD/'My Drive'/

In [ ]:
!ls 

GD  sample_data


In [ ]:
drive.mount('/GD',force_remount=True)

Mounted at /GD


In [ ]:
!cp -r /GD/'My Drive'/PetImages.zip .

In [ ]:
!ls 

GD  PetImages.zip  sample_data


In [ ]:
!unzip PetImages.zip


In [ ]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers

In [ ]:
!ls 

GD  PetImages  PetImages.zip  sample_data


In [ ]:
!mkdir PetImages/validation_data

In [ ]:
#!mv PetImages/Cat/ PetImages/train/Cat/

In [ ]:
#!mv PetImages/Dog/ PetImages/train/Dog/

In [ ]:
#!mkdir PetImages/test

In [ ]:
#!mkdir PetImages/test/Cat

In [ ]:
!mkdir PetImages/validation_data/Dog

In [ ]:
!mkdir PetImages/validation_data/Cat

In [ ]:
!ls PetImages/validation_data

Cat  Dog


## разделение данных на тестовое и обучающее множество

In [ ]:
import os, shutil
import random as rn

In [ ]:
train_dirs = ['PetImages/train/Dog', 
              'PetImages/train/Cat']

test_dirs =  ['PetImages/validation_data/Dog', 
              'PetImages/validation_data/Cat']

In [ ]:
# Страшный код

for i in range(len(train_dirs)):
    fnames = os.listdir(train_dirs[i])
    toCopy = len(fnames)//10

    filesToCopy = []
    for j in range(toCopy):
        filesToCopy.append(rn.choice(fnames))
    
    # cOPY
         
    for fi in filesToCopy:
        src = os.path.join(train_dirs[i],fi)
        dst = os.path.join(test_dirs[i], fi)
        shutil.copyfile(src,dst)

    
    for fi in filesToCopy:
        try:
            src = os.path.join(train_dirs[i],fi)
            os.remove(src)
            print(src+' deleted')
            #time.sleep(1)
        except:
            pass


PetImages/train/Dog/10960.jpg deleted
PetImages/train/Dog/3708.jpg deleted
PetImages/train/Dog/11280.jpg deleted
PetImages/train/Dog/8244.jpg deleted
PetImages/train/Dog/7277.jpg deleted
PetImages/train/Dog/7190.jpg deleted
PetImages/train/Dog/10375.jpg deleted
PetImages/train/Dog/6884.jpg deleted
PetImages/train/Dog/7529.jpg deleted
PetImages/train/Dog/5693.jpg deleted
PetImages/train/Dog/7570.jpg deleted
PetImages/train/Dog/10831.jpg deleted
PetImages/train/Dog/11861.jpg deleted
PetImages/train/Dog/9085.jpg deleted
PetImages/train/Dog/9049.jpg deleted
PetImages/train/Dog/10930.jpg deleted
PetImages/train/Dog/7380.jpg deleted
PetImages/train/Dog/7100.jpg deleted
PetImages/train/Dog/668.jpg deleted
PetImages/train/Dog/8583.jpg deleted
PetImages/train/Dog/1447.jpg deleted
PetImages/train/Dog/6557.jpg deleted
PetImages/train/Dog/5355.jpg deleted
PetImages/train/Dog/7714.jpg deleted
PetImages/train/Dog/3981.jpg deleted
PetImages/train/Dog/5023.jpg deleted
PetImages/train/Dog/7267.jpg dele

In [ ]:
# осмотр получившихся директорий

In [ ]:
!ls PetImages/validation_data/Dog

## подготовка генераторов

In [ ]:
# Воспоминание о генераторах

In [ ]:
def int_generator ():
    i = 0
    while True:
        yield i
        i += 1
        

In [ ]:
j = 0
for item in int_generator():
    print (item)
    j += 1
    if j > 10:
        break

0
1
2
3
4
5
6
7
8
9
10


In [ ]:
# взрослый генератор

In [ ]:
Pic_Y, Pic_X = 180, 180
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
                            
train_generator = train_datagen.flow_from_directory('PetImages/train',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 18508 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
                            
test_generator = test_datagen.flow_from_directory('PetImages/test',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 4542 images belonging to 2 classes.


In [ ]:
# Обучение модели с 0

model = models.Sequential()

model.add(layers.Conv2D(filters=32, kernel_size=3, activation="relu", input_shape = (Pic_Y, Pic_X, 3)))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=64, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=128, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))

model.add(layers.Flatten())
model.add(layers.Dense(1, activation="sigmoid"))


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 89, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 43, 43, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 41, 41, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 20, 20, 128)      0

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
# записывает лучшую модель
callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.h5",
        save_best_only=True,
        monitor="val_accuracy")]

In [ ]:
history = model.fit(train_generator, epochs=3, validation_data=test_generator,
    callbacks=callbacks)

Epoch 1/3
290/290 [==============================] - ETA: 0s - loss: 0.6429 - accuracy: 0.6182

/usr/local/lib/python3.9/dist-packages/PIL/TiffImagePlugin.py:819: UserWarning: Truncated File Read
  warnings.warn(str(msg))


290/290 [==============================] - 111s 335ms/step - loss: 0.6429 - accuracy: 0.6182 - val_loss: 0.5446 - val_accuracy: 0.7204
Epoch 2/3
290/290 [==============================] - 95s 328ms/step - loss: 0.5134 - accuracy: 0.7497 - val_loss: 0.4475 - val_accuracy: 0.7968
Epoch 3/3
290/290 [==============================] - 93s 322ms/step - loss: 0.4248 - accuracy: 0.8077 - val_loss: 0.4454 - val_accuracy: 0.8089


In [ ]:
!ls

convnet_from_scratch.h5  GD  PetImages	PetImages.zip  sample_data


In [ ]:
# Копирование сохраненой модели на гугл диск.
!cp convnet_from_scratch.h5 /GD/'My Drive'/convnet_from_scratch.h5

## Добавляем расширение данных 

In [ ]:
model_2 = models.Sequential()

model_2.add(layers.Conv2D(filters=32, kernel_size=3, activation="relu", input_shape = (Pic_Y, Pic_X, 3)))
model_2.add(layers.MaxPooling2D(pool_size=2))

model_2.add(layers.Conv2D(filters=64, kernel_size=3, activation="relu"))
model_2.add(layers.MaxPooling2D(pool_size=2))

model_2.add(layers.Conv2D(filters=128, kernel_size=3, activation="relu"))
model_2.add(layers.MaxPooling2D(pool_size=2))

model_2.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))
model_2.add(layers.MaxPooling2D(pool_size=2))
model_2.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))

model_2.add(layers.Flatten())
model_2.add(layers.Dense(1, activation="sigmoid"))


In [ ]:
model_2.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
# Новый интересный объект
callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath="augmentation_model.h5",
        save_best_only=True,
        monitor="val_loss")]

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 25,
                                   shear_range = 0.15,
                                   zoom_range = 0.3,
                                   fill_mode = 'nearest')
                            
train_generator = train_datagen.flow_from_directory('PetImages/train',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 18508 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
                            
test_generator = test_datagen.flow_from_directory('PetImages/test',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 4542 images belonging to 2 classes.


In [ ]:
history = model_2.fit(train_generator, epochs=5, validation_data=test_generator,
    callbacks=callbacks)

Epoch 1/5
290/290 [==============================] - ETA: 0s - loss: 0.6877 - accuracy: 0.5591

/usr/local/lib/python3.9/dist-packages/PIL/TiffImagePlugin.py:819: UserWarning: Truncated File Read
  warnings.warn(str(msg))


290/290 [==============================] - 235s 802ms/step - loss: 0.6877 - accuracy: 0.5591 - val_loss: 0.6301 - val_accuracy: 0.6625
Epoch 2/5
290/290 [==============================] - 232s 799ms/step - loss: 0.6040 - accuracy: 0.6714 - val_loss: 0.6227 - val_accuracy: 0.6874
Epoch 3/5
290/290 [==============================] - 238s 822ms/step - loss: 0.5389 - accuracy: 0.7340 - val_loss: 0.4583 - val_accuracy: 0.7814
Epoch 4/5
290/290 [==============================] - 234s 806ms/step - loss: 0.4874 - accuracy: 0.7656 - val_loss: 0.4085 - val_accuracy: 0.8126
Epoch 5/5
290/290 [==============================] - 236s 812ms/step - loss: 0.4507 - accuracy: 0.7901 - val_loss: 0.3942 - val_accuracy: 0.8267


## Перенос знаний!

In [ ]:
# закгрузка сверточных слоев VGG16

conv_base = tf.keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3))


58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Заморозка сверточных слоев
conv_base.trainable = False

In [ ]:
# Конструируем модель
model_3 = models.Sequential()
model_3.add(conv_base)
model_3.add(layers.Flatten())
model_3.add(layers.Dense(1, activation = 'sigmoid'))

In [ ]:
model_3.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath="VGG_FIX_model.h5",
        save_best_only=True,
        monitor="val_loss")]

In [ ]:
# фитим на тех же генераторах то есть с расширением данных
history = model_3.fit(train_generator, epochs=5, validation_data=test_generator, 
    callbacks=callbacks)

Epoch 1/5
290/290 [==============================] - 271s 911ms/step - loss: 0.3849 - accuracy: 0.8234 - val_loss: 0.2166 - val_accuracy: 0.9060
Epoch 2/5
290/290 [==============================] - 242s 833ms/step - loss: 0.2880 - accuracy: 0.8770 - val_loss: 0.1983 - val_accuracy: 0.9157
Epoch 3/5
290/290 [==============================] - 259s 893ms/step - loss: 0.2650 - accuracy: 0.8871 - val_loss: 0.2100 - val_accuracy: 0.9124
Epoch 4/5
290/290 [==============================] - 241s 833ms/step - loss: 0.2485 - accuracy: 0.8921 - val_loss: 0.1877 - val_accuracy: 0.9240
Epoch 5/5
290/290 [==============================] - 241s 832ms/step - loss: 0.2417 - accuracy: 0.8993 - val_loss: 0.1986 - val_accuracy: 0.9194


In [ ]:
# Копирование сохраненой модели на гугл диск.
!cp VGG_FIX_model.h5 /GD/'My Drive'/VGG_FIX_model.h5

In [ ]:
from keras.preprocessing import image
import numpy as np
from PIL import Image

def predict_image(image_path):
    # Загрузка изображения и подготовка его для подачи в модели
    img = Image.open(image_path)
    img = img.resize((Pic_Y, Pic_X)) # изменение размера изображения
    img = np.array(img) / 255.0 # нормализация значений пикселей
    img = np.expand_dims(img, axis=0) # добавление измерения пакета
    
    # Загрузка каждой из трех моделей
    model1 = tf.keras.models.load_model('/content/convnet_from_scratch.h5')
    model2 = tf.keras.models.load_model('/content/augmentation_model.h5')
    model3 = tf.keras.models.load_model('/content/VGG_FIX_model.h5')
    

    # Предсказание на входном изображении 
    pred1 = model1.predict(img)
    pred2 = model2.predict(img)
    pred3 = model3.predict(img)
    
    # Принятие окончательного решения на основе предсказаний всех трех моделей
    final_pred = np.mean([pred1, pred2, pred3], axis=0)
    if final_pred > 0.5:
        return 'Dog'
    else:
        return 'Cat'

In [ ]:
predict_image('/content/PetImages/validation_data/Dog/1001.jpg')

1/1 [==============================] - 0s 91ms/step


1/1 [==============================] - 0s 97ms/step


1/1 [==============================] - 0s 161ms/step


'Dog'

In [ ]:
не брать

In [ ]:
## Fine Turning

In [ ]:
# Размораживаем 4 верхних свертки
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
    layer.trainable = False

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
              metrics=["accuracy"])

callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath="fine_tuning.h5",
        save_best_only=True,
        monitor="val_loss")]


In [ ]:
history = model.fit(train_generator, epochs=1, validation_data=test_generator, 
    callbacks=callbacks)

In [ ]:
## Оценка модели на проверочном множестве.
model = tf.keras.models.load_model("convnet_from_scratch.h5")
test_loss, test_acc = model.evaluate(train_generator)
print(f"Test accuracy: {test_acc:.3f}")

2022-07-14 17:06:59.306282: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-14 17:06:59.306355: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-14 17:06:59.306408: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sergey-home): /proc/driver/nvidia/version does not exist
2022-07-14 17:06:59.306970: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-14 17:07:00.607380: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 259555328 excee

  1/320 [..............................] - ETA: 3:45 - loss: 0.3602 - accuracy: 0.9062

2022-07-14 17:07:01.352816: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 259555328 exceeds 10% of free system memory.


  2/320 [..............................] - ETA: 3:18 - loss: 0.3207 - accuracy: 0.8828

2022-07-14 17:07:02.060701: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 259555328 exceeds 10% of free system memory.


  3/320 [..............................] - ETA: 3:31 - loss: 0.3080 - accuracy: 0.8906

2022-07-14 17:07:02.633421: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 259555328 exceeds 10% of free system memory.


  4/320 [..............................] - ETA: 3:21 - loss: 0.2993 - accuracy: 0.8945

2022-07-14 17:07:03.432723: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 259555328 exceeds 10% of free system memory.


320/320 [==============================] - 231s 721ms/step - loss: 0.3440 - accuracy: 0.8485
Test accuracy: 0.848


In [ ]:
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError
import os
i=0
pa = "PetImages/train/Cat"
path = Path(pa).rglob("*.jpg")
for img_p in path:
    try:
        img = PIL.Image.open(img_p)
    except PIL.UnidentifiedImageError:
            i+=1
            print(img_p, i)
            os.remove(img_p)

PetImages/train/Cat/666.jpg 1


In [ ]:
#PetImages/train/Dog/11702.jpg 1